# **Project Machine Learning Pipeline**

In [1]:
try:
  import colab
  !pip install --upgrade pip
except:
  pass

In [2]:
!pip install -U tfx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 21.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 33.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.5/7.5 MB 61.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.0/147.0 kB 19.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.7/135.7 kB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 9.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.8/14.8 MB 95.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.6/60.6 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 206.6/206.6 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.9/128.9 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 80.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
#import library
import tensorflow as tf
from tfx.components import CsvExampleGen, StatisticsGen, SchemaGen, ExampleValidator, Transform, Trainer, Tuner
from tfx.proto import example_gen_pb2
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
import os
import pandas as pd

## **Set variabel**

In [4]:
PIPELINE_NAME = "mental-pipeline"
SCHEMA_PIPELINE_NAME = "mental-tfdv-schema"

PIPELINE_ROOT = os.path.join('pipelines', PIPELINE_NAME)
METADATA_PATH = os.path.join('metadata', PIPELINE_NAME, 'metadata.db')
SERVING_MODEL_DIR = os.path.join('serving_model', PIPELINE_NAME)


In [5]:
mental = pd.read_csv("/content/data/mental_health.csv")
mental.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27977 entries, 0 to 27976
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    27977 non-null  object
 1   label   27977 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 437.3+ KB


In [6]:
data_path = "data"
mental.to_csv(os.path.join(data_path, "mental_health.csv"), index = False)

In [7]:
DATA_ROOT = 'data'

In [8]:
interactive_context = InteractiveContext(pipeline_root = PIPELINE_ROOT)

Tahapan pengolahan data ini terdiri dari beberapa proses seperti berikut.<br>

- Proses data ingestion. Pada proses ini menggunakan komponen ExampleGen.
- Proses data validation. Proses ini akan dibuat menggunakan beberapa komponen yang disediakan oleh TFX, seperti StatisticGen, SchemaGen, dan ExampleValidator
- Proses data preprocessing. Pada proses ini akan memanfaatkan komponen Transform.

## Membuat Tahapan Data Ingestion

Pada tahap ini akan melakukan data ingestion sebagai bagian awal dari pipeline tersebut.<BR>

 akan melakukan data ingestion menggunakan komponen CsvExampleGen() yang telah disediakan oleh TFX. Komponen ini dipilih karena dataset yang digunakan memiliki format CSV.

In [9]:
output = example_gen_pb2.Output(
    split_config = example_gen_pb2.SplitConfig(splits = [
        example_gen_pb2.SplitConfig.Split(name = "train", hash_buckets = 8),
        example_gen_pb2.SplitConfig.Split(name = "eval", hash_buckets = 2)
    ])
)

example_gen = CsvExampleGen(input_base = DATA_ROOT, output_config = output)

Pada ExampleGen diatas membagi dataset untuk proses training dan evaluasi menjadi rasio 80:20

In [10]:
interactive_context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen
    execution_id: 1
    outputs:
        examples: OutputChannel(artifact_type=Examples, producer_component_id=CsvExampleGen, output_key=examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## Membuat Tahapan Data Validation

- Membuat summary statistic
Pada latihan ini, dengan membuat summary statistic menggunakan komponen **StatisticsGen()**. Komponen memiliki parameter input berupa examples untuk menerima dataset dari komponen ExampleGen.

In [11]:
statistic_gen = StatisticsGen(
    examples = example_gen.outputs["examples"]
)

interactive_context.run(statistic_gen)

ExecutionResult(
    component_id: StatisticsGen
    execution_id: 2
    outputs:
        statistics: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=StatisticsGen, output_key=statistics, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

 menggunakan interactive_context.show() untuk menampilkan summary statistic yang telah dibuat.

In [12]:
interactive_context.show(statistic_gen.outputs['statistics'])

- Membuat data schema<br>
Setelah membuat summary statistic, perlu membuat data schema dengan menggunakan komponen **ShcemaGen()** untuk membuat data schema. Komponen ini akan menerima input berupa summary statistic.

In [13]:
schema_gen = SchemaGen(statistics = statistic_gen.outputs["statistics"])
interactive_context.run(schema_gen)

ExecutionResult(
    component_id: SchemaGen
    execution_id: 3
    outputs:
        schema: OutputChannel(artifact_type=Schema, producer_component_id=SchemaGen, output_key=schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Membuat SchemaGen untuk memperoleh informasi dari feature text dan label

In [14]:
interactive_context.show(schema_gen.outputs["schema"])

,Type,Presence,Valency,Domain
Feature name,,,,
'label',INT,required,,-
'text',BYTES,required,,-


Terlihat dari SchemaGen yang telah dibuat, terdapat dua feature yaitu label dan text, label disini untuk menentukan prediksi apakah text berupa kalimat yang menandakan mental dalam kondisi baik atau tidak, kemudian text merupakan inputan untuk menentukan label, label bertipe int dan text bertipe bytes dan masing-masing feature tersebut required untuk melakukan prediksi

- Mengidentifikasi anomali pada dataset<br>
 Proses ini dilakukan menggunakan komponen **ExampleValidator()** yang telah disediakan oleh TFX. Komponen ini membutuhkan keluaran dari statistics_gen dan schema_gen sebagai parameter input.

In [15]:
example_validator = ExampleValidator(
    statistics = statistic_gen.outputs["statistics"],
    schema = schema_gen.outputs["schema"]
)
interactive_context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator
    execution_id: 4
    outputs:
        anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=ExampleValidator, output_key=anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Untuk menampilkan hasil dari validasi dari komponen tersebut dengan menjalankan kode berikut.

In [16]:
interactive_context.show(example_validator.outputs["anomalies"])

Berdasarkan hasil validasi di atas, tidak terdapat anomali pada dataset yang kita gunakan pada latihan ini.

## Membuat Tahapan Data Preprocessing

Setelah melakukan data validation, tahap berikutnya adalah melakukan data preprocessing. Tujuan dari tahap ini adalah mengubah data mentah menjadi data yang siap digunakan untuk melatih model. Pada latihan ini, dengan menggunakan TFT dan komponen Transform untuk melakukan data preprocessing.<br>

Transform akan menerima inputan berupa module file yang berisi preprocessing function. <br>

In [17]:
TRANSFORM_MODULE_FILE = "mental_transform.py"

Membuat mental_transofrm.py untuk menyimpan proses transform pada module mental_transform.py

In [18]:
%%writefile {TRANSFORM_MODULE_FILE}

import tensorflow as tf

LABEL_KEY = "label"
FEATURE_KEY = "text"

def transformed_name(key) :
    return key + "_xf"

def preprocessing_fn(inputs) :
    outputs = {}
    outputs[transformed_name(LABEL_KEY)] = tf.cast(inputs[LABEL_KEY], tf.int64)
    outputs[transformed_name(FEATURE_KEY)] = tf.strings.lower(inputs[FEATURE_KEY])
    return outputs

Writing mental_transform.py


Kode di atas akan menghasilkan sebuah module dengan nama **sarcasm_transform.py**. Module ini berisi dua buah function, yaitu transformed_name() dan preprocessing_fn().

- function transformed_name() digunakan untuk mengubah nama fitur yang telah di-transform. Perubahan ini dilakukan dengan menambahkan **“_xf”** pada nama fitur tersebut.

- Pada function preprocessing_fn(), hanya menerapkan tahapan preprocessing yang sederhana. Tahapan yang dilakukan hanya mengubah data yang terdapat dalam fitur “headline” ke dalam bentuk *lowercase*.

Setelah membuat module yang berisi preprocessing function, tahap selanjutnya adalah mendefinisikan komponen Transform(). Komponen ini akan menerima input berupa *examples* untuk menerima dataset dari komponen ExampleGen, schema untuk menerima data schema dari SchemaGen, dan *module_file* untuk menerima module file yang berisi preprocessing function.

In [19]:
transform = Transform(
    examples = example_gen.outputs["examples"],
    schema = schema_gen.outputs['schema'],
    module_file = os.path.abspath(TRANSFORM_MODULE_FILE)
)

interactive_context.run(transform)

ExecutionResult(
    component_id: Transform
    execution_id: 5
    outputs:
        transform_graph: OutputChannel(artifact_type=TransformGraph, producer_component_id=Transform, output_key=transform_graph, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        transformed_examples: OutputChannel(artifact_type=Examples, producer_component_id=Transform, output_key=transformed_examples, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        updated_analyzer_cache: OutputChannel(artifact_type=TransformCache, producer_component_id=Transform, output_key=updated_analyzer_cache, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=pre_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        pre_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=pre_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_schema: OutputChannel(artifact_type=Schema, producer_component_id=Transform, output_key=post_transform_schema, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_stats: OutputChannel(artifact_type=ExampleStatistics, producer_component_id=Transform, output_key=post_transform_stats, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        post_transform_anomalies: OutputChannel(artifact_type=ExampleAnomalies, producer_component_id=Transform, output_key=post_transform_anomalies, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Berdasarkan gambar di atas, dapat dilihat bahwa komponen Transform akan menghasilkan beberapa output mulai dari **transform_graph**, **transformed_examples** (transformed data), dll. Keseluruhan output nantinya digunakan untuk melakukan preprocessing terhadap serving set. Hal ini akan membantu untuk memastikan preprocessing yang dilakukan pada fase training sama dengan fase deployment sehingga dapat mencegah feature skew.

## **Model Development**

Membuat mental_trainer.py untuk menyimpan proses train pada module mental_trainer.py<br>

Membuat proses untuk membuat model machine learning dan training model

In [20]:
#Membuat file module stress_trainer.py
%%writefile {TRAINER_MODULE_FILE}

#Import library
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import os
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "label"
FEATURE_KEY = "text"

#Rename feature yang ditransform
def transformed_name(key) :
    return key + "_xf"

#Load data yang dicompress dalam TFRecord
def gzip_reader_fn(filenames) :
    return tf.data.TFRecordDataset(filenames, compression_type = 'GZIP')

#Membuat transformed_feature dan batch data
def input_fn(file_pattern, tf_transform_output, num_epochs,
            batch_size=64)->tf.data.Dataset:

            transform_feature_spec = (
                tf_transform_output.transformed_feature_spec().copy()
            )

            dataset = tf.data.experimental.make_batched_features_dataset(
                file_pattern = file_pattern,
                batch_size = batch_size,
                features = transform_feature_spec,
                reader = gzip_reader_fn,
                num_epochs = num_epochs,
                label_key = transformed_name(LABEL_KEY)
            )
            return dataset

#Membuat arsitektur model
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

#Melakukan tokenisasi, dan standarisasi data
vectorize_layer = layers.TextVectorization(
    standardize = "lower_and_strip_punctuation",
    max_tokens = VOCAB_SIZE,
    output_mode = 'int',
    output_sequence_length = SEQUENCE_LENGTH
)

embedding_dim = 16

#Membuat model machine learning
def model_builder() :
    #Machine Learning Model
    inputs = tf.keras.Input(shape=(1,), name = transformed_name(FEATURE_KEY), dtype = tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name = "embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation = "relu")(x)
    x = layers.Dense(32, activation = "relu")(x)
    outputs = layers.Dense(1, activation = "sigmoid")(x)

    model = tf.keras.Model(inputs = inputs, outputs = outputs)

    #Model compile menggunakan binary_crossentropy, Adam dengan learning_rate 0.01 dan metrics BinaryAccuray
    model.compile(
        loss = 'binary_crossentropy',
        optimizer = tf.keras.optimizers.Adam(0.01),
        metrics = [tf.keras.metrics.BinaryAccuracy()]
    )

    model.summary()
    return model

#Melakukan preprocessing pada raw request data yang nanti akan digunakan pada proses deployment
def _get_serve_tf_examples_fn(model, tf_transform_output) :

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples) :
        feature_spec = tf_transform_output.raw_feature_spec()
        feature_spec.pop(LABEL_KEY)
        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)
        transformed_features = model.tft_layer(parsed_features)
        return model(transformed_features)
    return serve_tf_examples_fn

#Melakukan proses training
def run_fn(fn_args : FnArgs) -> None :
    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq = 'batch'
    )

    #Callback untuk EarlyStopping dan ModelCheckpoint
    es = tf.keras.callbacks.EarlyStopping(monitor = 'val_binary_accuracy', mode = 'max', verbose = 1, patience = 10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor = 'val_binary_accuracy', mode = 'max', verbose=1, save_best_only=True)

    #Load output dari transform data
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    #Membuat batch data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]
        ]
    )

    #Build model
    model = model_builder()

    #Training model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000,
            validation_steps = 1000,
            epochs = 10
    )

    signatures = {
        'serving_default' :
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                tf.TensorSpec(
                                    shape = [None],
                                    dtype = tf.string,
                                    name = 'examples'
                                )
        )
    }

    #Menyimpan model pada serving_model_dir
    model.save(fn_args.serving_model_dir, save_format = 'tf', signatures = signatures)

Writing {TRAINER_MODULE_FILE}


Seluruh fungsi di atas, akan disatukan ke dalam sebuah module file. Namun, sebelum membuat module file,  perlu mendefinisikan nama dari module tersebut.

In [21]:
TRAINER_MODULE_FILE = "mental_trainer.py"

Selanjutnya, buatlah sebuah module dengan kode berikut.

In [22]:
%%writefile {TRAINER_MODULE_FILE}
import tensorflow as tf
import tensorflow_transform as tft
from tensorflow.keras import layers
import os
import tensorflow_hub as hub
from tfx.components.trainer.fn_args_utils import FnArgs

LABEL_KEY = "label"
FEATURE_KEY = "text"

def transformed_name(key):
    """Renaming transformed features"""
    return key + "_xf"

def gzip_reader_fn(filenames):
    """Loads compressed data"""
    return tf.data.TFRecordDataset(filenames, compression_type='GZIP')


def input_fn(file_pattern,
             tf_transform_output,
             num_epochs,
             batch_size=64)->tf.data.Dataset:
    """Get post_tranform feature & create batches of data"""

    # Get post_transform feature spec
    transform_feature_spec = (
        tf_transform_output.transformed_feature_spec().copy())

    # create batches of data
    dataset = tf.data.experimental.make_batched_features_dataset(
        file_pattern=file_pattern,
        batch_size=batch_size,
        features=transform_feature_spec,
        reader=gzip_reader_fn,
        num_epochs=num_epochs,
        label_key = transformed_name(LABEL_KEY))
    return dataset

# os.environ['TFHUB_CACHE_DIR'] = '/hub_chace'
# embed = hub.KerasLayer("https://tfhub.dev/google/universal-sentence-encoder/4")

# Vocabulary size and number of words in a sequence.
VOCAB_SIZE = 10000
SEQUENCE_LENGTH = 100

vectorize_layer = layers.TextVectorization(
    standardize="lower_and_strip_punctuation",
    max_tokens=VOCAB_SIZE,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)


embedding_dim=16
def model_builder():
    """Build machine learning model"""
    inputs = tf.keras.Input(shape=(1,), name=transformed_name(FEATURE_KEY), dtype=tf.string)
    reshaped_narrative = tf.reshape(inputs, [-1])
    x = vectorize_layer(reshaped_narrative)
    x = layers.Embedding(VOCAB_SIZE, embedding_dim, name="embedding")(x)
    x = layers.GlobalAveragePooling1D()(x)
    x = layers.Dense(64, activation='relu')(x)
    x = layers.Dense(32, activation="relu")(x)
    outputs = layers.Dense(1, activation='sigmoid')(x)


    model = tf.keras.Model(inputs=inputs, outputs = outputs)

    model.compile(
        loss = 'binary_crossentropy',
        optimizer=tf.keras.optimizers.Adam(0.01),
        metrics=[tf.keras.metrics.BinaryAccuracy()]

    )

    # print(model)
    model.summary()
    return model


def _get_serve_tf_examples_fn(model, tf_transform_output):

    model.tft_layer = tf_transform_output.transform_features_layer()

    @tf.function
    def serve_tf_examples_fn(serialized_tf_examples):

        feature_spec = tf_transform_output.raw_feature_spec()

        feature_spec.pop(LABEL_KEY)

        parsed_features = tf.io.parse_example(serialized_tf_examples, feature_spec)

        transformed_features = model.tft_layer(parsed_features)

        # get predictions using the transformed features
        return model(transformed_features)

    return serve_tf_examples_fn

def run_fn(fn_args: FnArgs) -> None:

    log_dir = os.path.join(os.path.dirname(fn_args.serving_model_dir), 'logs')

    tensorboard_callback = tf.keras.callbacks.TensorBoard(
        log_dir = log_dir, update_freq='batch'
    )

    es = tf.keras.callbacks.EarlyStopping(monitor='val_binary_accuracy', mode='max', verbose=1, patience=10)
    mc = tf.keras.callbacks.ModelCheckpoint(fn_args.serving_model_dir, monitor='val_binary_accuracy', mode='max', verbose=1, save_best_only=True)


    # Load the transform output
    tf_transform_output = tft.TFTransformOutput(fn_args.transform_graph_path)

    # Create batches of data
    train_set = input_fn(fn_args.train_files, tf_transform_output, 10)
    val_set = input_fn(fn_args.eval_files, tf_transform_output, 10)
    vectorize_layer.adapt(
        [j[0].numpy()[0] for j in [
            i[0][transformed_name(FEATURE_KEY)]
                for i in list(train_set)]])

    # Build the model
    model = model_builder()


    # Train the model
    model.fit(x = train_set,
            validation_data = val_set,
            callbacks = [tensorboard_callback, es, mc],
            steps_per_epoch = 1000,
            validation_steps= 1000,
            epochs=10)
    signatures = {
        'serving_default':
        _get_serve_tf_examples_fn(model, tf_transform_output).get_concrete_function(
                                    tf.TensorSpec(
                                    shape=[None],
                                    dtype=tf.string,
                                    name='examples'))
    }
    model.save(fn_args.serving_model_dir, save_format='tf', signatures=signatures)

Writing mental_trainer.py


Kode di atas akan menghasilkan sebuah module dengan nama mental_trainer.py. Module tersebut mengandung training function beserta beberapa helper function yang telah disebutkan sebelumnya.<br>

Tahap selanjutnya adalah mendefinisikan komponen Trainer(). Komponen ini akan menerima beberapa input seperti berikut. <br>

- **module_file** untuk menerima module file yang mengandung mengandung training function beserta beberapa helper function.
- **examples** untuk menerima dataset dari komponen ExampleGen.
- **schema** untuk menerima data schema dari komponen SchemaGen.
- **tansform_graph** untuk menerima transform graph yang dihasilkan dari komponen Transform.
- **train_args** untuk menampung parameter training.
- **eval_args** untuk menampung parameter testing atau evaluation.

In [23]:
from tfx.proto import trainer_pb2

trainer = Trainer(
    module_file = os.path.abspath(TRAINER_MODULE_FILE),
    examples = transform.outputs['transformed_examples'],
    transform_graph = transform.outputs['transform_graph'],
    schema = schema_gen.outputs['schema'],
    train_args = trainer_pb2.TrainArgs(splits = ['train']),
    eval_args = trainer_pb2.EvalArgs(splits = ['eval'])
)

interactive_context.run(trainer)

Instructions for updating:
Use `tf.data.Dataset.map(tf.io.parse_example(...))` instead.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 tf.reshape (TFOpLambda)     (None,)                   0         
                                                                 
 text_vectorization (TextVe  (None, 100)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 100, 16)           160000    
                                                                 
 global_average_pooling1d (  (None, 16)                0         
 GlobalAveragePooling1D)                                         
                                                                 
 dense (Dense)               (None, 64)                1088  


Epoch 1: val_binary_accuracy improved from -inf to 0.90984, saving model to pipelines/mental-pipeline/Trainer/model/6/Format-Serving
1000/1000 [==============================] - 18s 16ms/step - loss: 0.1915 - binary_accuracy: 0.9222 - val_loss: 0.2597 - val_binary_accuracy: 0.9098
Epoch 2/10
 998/1000 [============================>.] - ETA: 0s - loss: 0.0804 - binary_accuracy: 0.9689

1000/1000 [==============================] - 5s 5ms/step - loss: 0.0804 - binary_accuracy: 0.9689
Epoch 3/10
 994/1000 [============================>.] - ETA: 0s - loss: 0.0484 - binary_accuracy: 0.9809

1000/1000 [==============================] - 5s 5ms/step - loss: 0.0484 - binary_accuracy: 0.9809
Epoch 4/10
 500/1000 [==============>...............] - ETA: 2s - loss: 0.0447 - binary_accuracy: 0.9834

1000/1000 [==============================] - 2s 2ms/step - loss: 0.0447 - binary_accuracy: 0.9834


ExecutionResult(
    component_id: Trainer
    execution_id: 6
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Trainer, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_run: OutputChannel(artifact_type=ModelRun, producer_component_id=Trainer, output_key=model_run, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

## **Membuat Tahapan Analisis dan Validasi Model**

- Komponen Resolver<br>
Untuk melakukan analisis dan validasi model, perlu menyediakan sebuah baseline model. Hal ini sangat penting terutama ketika memiliki lebih dari satu versi model dan ingin membandingkan dua buah versi model yang berbeda. Untuk melakukannya, kita bisa memanfaatkan komponen **Resolver()**.

In [24]:
from tfx.dsl.components.common.resolver import Resolver
from tfx.dsl.input_resolution.strategies.latest_blessed_model_strategy import LatestBlessedModelStrategy
from tfx.types import Channel
from tfx.types.standard_artifacts import Model, ModelBlessing

model_resolver = Resolver(
    strategy_class= LatestBlessedModelStrategy,
    model = Channel(type=Model),
    model_blessing = Channel(type=ModelBlessing)
).with_id('Latest_blessed_model_resolver')

interactive_context.run(model_resolver)

ExecutionResult(
    component_id: Latest_blessed_model_resolver
    execution_id: 7
    outputs:
        model: OutputChannel(artifact_type=Model, producer_component_id=Latest_blessed_model_resolver, output_key=model, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        model_blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Latest_blessed_model_resolver, output_key=model_blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

- Komponen Evaluator<br>
Setelah mendefinisikan komponen Resolver, tahap selanjutnya adalah membuat beberapa konfigurasi untuk mengevaluasi model. Konfigurasi ini dibuat menggunakan library TFMA seperti berikut.

In [25]:
import tensorflow_model_analysis as tfma

eval_config = tfma.EvalConfig(
    model_specs=[tfma.ModelSpec(label_key='label')],
    slicing_specs=[tfma.SlicingSpec()],
    metrics_specs=[
        tfma.MetricsSpec(metrics=[

            tfma.MetricConfig(class_name='ExampleCount'),
            tfma.MetricConfig(class_name='AUC'),
            tfma.MetricConfig(class_name='FalsePositives'),
            tfma.MetricConfig(class_name='TruePositives'),
            tfma.MetricConfig(class_name='FalseNegatives'),
            tfma.MetricConfig(class_name='TrueNegatives'),
            tfma.MetricConfig(class_name='BinaryAccuracy',
                threshold=tfma.MetricThreshold(
                    value_threshold=tfma.GenericValueThreshold(
                        lower_bound={'value':0.5}),
                    change_threshold=tfma.GenericChangeThreshold(
                        direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                        absolute={'value':0.0001})
                    )
            )
        ])
    ]

)

Pada contoh kode di atas telah mengatur beberapa konfigurasi seperti metrik untuk mengevaluasi model beserta nilai threshold dari suatu metrik.

Setelah membuat konfigurasi yang akan digunakan untuk mengevaluasi model, tahap berikutnya adalah mendefinisikan komponen Evaluator. Komponen ini akan menerima beberapa input seperti berikut.

- examples untuk menerima dataset dari komponen ExampleGen.<br>

- model untuk menerima model yang dihasilkan dari komponen Trainer.

- baseline_model untuk menampung baseline model yang disediakan oleh komponen Resolver.

- eval_config untuk menerima konfigurasi untuk mengevaluasi model.

In [26]:
from tfx.components import Evaluator

evaluator = Evaluator(
    examples = example_gen.outputs['examples'],
    model = trainer.outputs['model'],
    baseline_model = model_resolver.outputs['model'],
    eval_config = eval_config
)

interactive_context.run(evaluator)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


ExecutionResult(
    component_id: Evaluator
    execution_id: 8
    outputs:
        evaluation: OutputChannel(artifact_type=ModelEvaluation, producer_component_id=Evaluator, output_key=evaluation, additional_properties={}, additional_custom_properties={}, _input_trigger=None
        blessing: OutputChannel(artifact_type=ModelBlessing, producer_component_id=Evaluator, output_key=blessing, additional_properties={}, additional_custom_properties={}, _input_trigger=None)

Hasil evaluasi divisualisasikan menggunakan library TFMA seperti contoh kode di bawah ini.

In [27]:
eval_result = evaluator.outputs['evaluation'].get()[0].uri
tfma_result = tfma.load_eval_result(eval_result)
tfma.view.render_slicing_metrics(tfma_result)
tfma.addons.fairness.view.widget_view.render_fairness_indicator(
    tfma_result
)

### **Komponen Pusher**

In [31]:
from tfx.components import Pusher
from tfx.proto import pusher_pb2

pusher = Pusher(
    model = trainer.outputs['model'],
    model_blessing = evaluator.outputs['blessing'],
    push_destination = pusher_pb2.PushDestination(
        filesystem = pusher_pb2.PushDestination.Filesystem(
            base_directory = "serving_model_dir/mental-prediction-model"
        )
    )
)

interactive_context.run(pusher)

ExecutionResult(
    component_id: Pusher
    execution_id: 9
    outputs:
        pushed_model: OutputChannel(artifact_type=PushedModel, producer_component_id=Pusher, output_key=pushed_model, additional_properties={}, additional_custom_properties={}, _input_trigger=None)